<a href="https://colab.research.google.com/github/bdmarvin1/glitter-ads/blob/main/Google_Ads_Trend_Reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# SECTION 1: INSTALL REQUIRED LIBRARIES
# ==============================================================================
!pip install google-ads google-auth-oauthlib gspread python-dateutil google-api-python-client

# ==============================================================================
# SECTION 2: IMPORTS & AUTHENTICATION
# ==============================================================================
import gspread
from gspread.utils import rowcol_to_a1
from google.colab import auth, userdata
from google.auth import default
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException
import json
import time
import datetime
from dateutil.relativedelta import relativedelta
# --- NEW: Import for Google Docs API ---
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# --- UPDATED: Authenticate with all required scopes (Sheets, Drive, Docs) ---
auth.authenticate_user()
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents']
creds, _ = default(scopes=SCOPES)
gc = gspread.authorize(creds)


# --- Google Ads API Configuration ---
google_ads_client = None
try:
    print("--- Initializing Google Ads Client ---")
    credentials = {
        "developer_token": userdata.get('ADS_DEVELOPER_TOKEN'),
        "client_id": userdata.get('ADS_CLIENT_ID'),
        "client_secret": userdata.get('ADS_CLIENT_SECRET'),
        "refresh_token": userdata.get('ADS_REFRESH_TOKEN'),
        "login_customer_id": userdata.get('ADS_LOGIN_CUSTOMER_ID'),
        "use_proto_plus": True
    }
    google_ads_client = GoogleAdsClient.load_from_dict(credentials)
    print("✅ Google Ads client initialized successfully!")
except Exception as e:
    print(f"\n❌ An unexpected error occurred during setup: {e}")


# --- NEW: Google Docs API Configuration ---
DOCS_LOG_ID = '1ZMv52EXlCDNX0sWBDcQRNdvuOxRcdOogMy0jHGvVgTE'
docs_service = None
try:
    print("--- Initializing Google Docs Client ---")
    docs_service = build('docs', 'v1', credentials=creds)
    print("✅ Google Docs client initialized successfully!")
except HttpError as e:
    print(f"\n❌ An error occurred during Google Docs client setup: {e}")
except Exception as e:
    print(f"\n❌ An unexpected error occurred during setup: {e}")


# ==============================================================================
# SECTION 3: CONFIGURATION
# ==============================================================================
KEYWORD_GROUPS = {
    'General Storage': ["self storage", "self storage unit", "self storage units", "storage", "storage facilities", "storage facility", "storage for rent", "storage locker", "storage unit", "storage units"],
    'Near Me': ["storage units near me", "storage unit near me", "storage facility near me", "self storage near me", "business storage near me", "vehicle storage near me", "boat storage near me", "rv storage near me", "car storage near me", "climate controlled storage near me"],
    'Features': ["drive up storage units", "drive up storage", "drive up storage facility", "climate controlled storage", "climate controlled storage unit", "climate controlled storage units", "heated storage", "heated storage units", "temperature control storage", "temperature controlled storage", "covered boat storage", "covered rv storage", "mini storage", "indoor storage units", "indoor storage", "indoor self storage", "outdoor storage", "outdoor storage units", "24 hour storage"],
    'Customer Types': ["business storage", "construction storage", "contractor storage", "commercial storage", "commercial storage units", "military storage", "student storage", "wine storage", "affordable storage", "cheap storage units"],
    'Vehicle Storage': ["auto storage", "boat parking", "boat storage", "car storage", "camper storage", "motorcycle storage", "rv storage", "truck storage", "trailer parking", "trailer storage", "vehicle storage", "5x5 storage unit", "10x10 storage unit", "10x15 storage unit", "10x20 storage unit"]
}
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/13QJZGvR8Zp_floekTod4V3LvLZKLfKdCLFjCKRhU2tQ/edit?gid=0#gid=0'
US_STATES = [
  "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

US_CITIES = [
  "New York, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", "Phoenix, AZ", "Philadelphia, PA", "San Antonio, TX", "San Diego, CA", "Dallas, TX", "Jacksonville, FL", "Fort Worth, TX", "San Jose, CA", "Austin, TX", "Charlotte, NC", "Columbus, OH", "Indianapolis, IN", "San Francisco, CA", "Seattle, WA", "Denver, CO", "Oklahoma City, OK", "Nashville, TN", "Washington, DC", "El Paso, TX", "Las Vegas, NV", "Boston, MA", "Detroit, MI", "Louisville, KY", "Portland, OR", "Memphis, TN", "Baltimore, MD", "Milwaukee, WI", "Albuquerque, NM", "Tucson, AZ", "Fresno, CA", "Sacramento, CA", "Atlanta, GA", "Mesa, AZ", "Kansas City, MO", "Raleigh, NC", "Colorado Springs, CO", "Omaha, NE", "Miami, FL", "Virginia Beach, VA", "Long Beach, CA", "Oakland, CA", "Minneapolis, MN", "Bakersfield, CA", "Tulsa, OK", "Tampa, FL", "Arlington, TX", "Aurora, CO", "Wichita, KS", "Cleveland, OH", "New Orleans, LA", "Henderson, NV", "Honolulu, HI", "Anaheim, CA", "Orlando, FL", "Lexington, KY", "Stockton, CA", "Riverside, CA", "Irvine, CA", "Corpus Christi, TX", "Newark, NJ", "Santa Ana, CA", "Cincinnati, OH", "Pittsburgh, PA", "Saint Paul, MN", "Greensboro, NC", "Jersey City, NJ", "Durham, NC", "Lincoln, NE", "North Las Vegas, NV", "Plano, TX", "Anchorage, AK", "Gilbert, AZ", "Madison, WI", "Reno, NV", "Chandler, AZ", "St. Louis, MO", "Chula Vista, CA", "Buffalo, NY", "Fort Wayne, IN", "Lubbock, TX", "St. Petersburg, FL", "Toledo, OH", "Laredo, TX", "Port St. Lucie, FL", "Glendale, AZ", "Irving, TX", "Winston-Salem, NC", "Chesapeake, VA", "Garland, TX", "Scottsdale, AZ", "Boise, ID", "Hialeah, FL", "Frisco, TX", "Richmond, VA", "Cape Coral, FL", "Norfolk, VA", "Spokane, WA", "Huntsville, AL", "Santa Clarita, CA", "Tacoma, WA", "Fremont, CA", "McKinney, TX", "San Bernardino, CA", "Baton Rouge, LA", "Modesto, CA", "Fontana, CA", "Salt Lake City, UT", "Moreno Valley, CA", "Des Moines, IA", "Worcester, MA", "Yonkers, NY", "Fayetteville, NC", "Sioux Falls, SD", "Grand Prairie, TX", "Rochester, NY", "Tallahassee, FL", "Little Rock, AR", "Amarillo, TX", "Overland Park, KS", "Columbus, GA", "Augusta, GA", "Mobile, AL", "Oxnard, CA", "Grand Rapids, MI", "Peoria, AZ", "Vancouver, WA", "Knoxville, TN", "Birmingham, AL", "Montgomery, AL", "Providence, RI", "Huntington Beach, CA", "Brownsville, TX", "Chattanooga, TN", "Fort Lauderdale, FL", "Tempe, AZ", "Akron, OH", "Glendale, CA", "Clarksville, TN", "Ontario, CA", "Newport News, VA", "Elk Grove, CA", "Cary, NC", "Aurora, IL", "Salem, OR", "Pembroke Pines, FL", "Eugene, OR", "Santa Rosa, CA", "Rancho Cucamonga, CA", "Shreveport, LA", "Garden Grove, CA", "Oceanside, CA", "Fort Collins, CO", "Springfield, MO", "Murfreesboro, TN", "Surprise, AZ", "Lancaster, CA", "Denton, TX", "Roseville, CA", "Palmdale, CA", "Corona, CA", "Salinas, CA", "Killeen, TX", "Paterson, NJ", "Alexandria, VA", "Hollywood, FL", "Hayward, CA", "Charleston, SC", "Macon, GA", "Lakewood, CO", "Sunnyvale, CA", "Kansas City, KS", "Springfield, MA", "Bellevue, WA", "Naperville, IL", "Joliet, IL", "Bridgeport, CT", "Mesquite, TX", "Pasadena, TX", "Olathe, KS", "Escondido, CA", "Savannah, GA", "McAllen, TX", "Gainesville, FL", "Pomona, CA", "Rockford, IL", "Thornton, CO", "Waco, TX", "Visalia, CA", "Syracuse, NY", "Columbia, SC", "Midland, TX", "Miramar, FL", "Palm Bay, FL", "Lakewood,NJ", "Jackson, MS", "Coral Springs, FL", "Victorville, CA", "Elizabeth, NJ", "Fullerton, CA", "Meridian, ID", "Torrance, CA", "Stamford, CT", "West Valley City, UT", "Orange, CA", "Cedar Rapids, IA", "Warren, MI", "Hampton, VA", "New Haven, CT", "Pasadena, CA", "Kent, WA", "Dayton, OH", "Fargo, ND", "Lewisville, TX", "Carrollton, TX", "Round Rock, TX", "Sterling Heights, MI", "Santa Clara, CA", "Norman, OK", "Columbia, MO", "Abilene, TX", "Pearland, TX", "Athens, GA", "College Station, TX", "Clovis, CA", "West Palm Beach, FL", "Allentown, PA", "North Charleston, SC", "Simi Valley, CA", "Topeka, KS", "Wilmington, NC", "Lakeland, FL", "Thousand Oaks, CA", "Concord, CA", "Rochester, MN", "Vallejo, CA", "Ann Arbor, MI", "Broken Arrow, OK", "Fairfield, CA", "Lafayette, LA", "Hartford, CT", "Arvada, CO", "Berkeley, CA", "Independence, MO", "Billings, MT", "Cambridge, MA", "Lowell, MA", "Odessa, TX", "High Point, NC", "League City, TX", "Antioch, CA", "Richardson, TX", "Goodyear, AZ", "Pompano Beach, FL", "Nampa, ID", "Menifee, CA", "Las Cruces, NM", "Clearwater, FL", "West Jordan, UT", "New Braunfels, TX", "Manchester, NH", "Miami Gardens, FL", "Waterbury, CT", "Provo, UT", "Evansville, IN", "Richmond, CA", "Westminster, CO", "Elgin, IL", "Conroe, TX", "Greeley, CO", "Lansing, MI", "Buckeye, AZ", "Tuscaloosa, AL", "Allen, TX", "Carlsbad, CA", "Everett, WA", "Springfield,IL", "Beaumont, TX", "Murrieta, CA", "Rio Rancho, NM", "Temecula, CA", "Concord, NC", "Tyler, TX", "Davie, FL", "South Fulton, GA", "Peoria, IL", "Sparks, NV", "Gresham, OR", "Santa Maria, CA", "Pueblo, CO", "Hillsboro, OR", "Edison, NJ", "Sugar Land, TX", "Ventura, CA", "Downey, CA", "Costa Mesa, CA", "Centennial, CO", "Edinburg, TX", "Spokane Valley, WA", "Jurupa Valley, CA", "Bend, OR", "West Covina, CA", "Boulder, CO", "Palm Coast, FL", "Lee's Summit, MO", "Dearborn, MI", "Green Bay, WI", "St. George, UT", "Woodbridge, NJ", "Brockton, MA", "Renton, WA", "Sandy Springs, GA", "Rialto, CA", "El Monte, CA", "Vacaville, CA", "Fishers, IN", "South Bend, IN", "Carmel, IN", "Yuma, AZ", "Burbank, CA", "Lynn, MA", "Quincy, MA", "El Cajon, CA", "Fayetteville, AR", "Suffolk, VA", "San Mateo, CA", "Chico, CA", "Inglewood, CA", "Wichita Falls, TX", "Boca Raton, FL", "Hesperia, CA", "Daly City, CA", "Clinton, MI", "Georgetown, TX", "New Bedford, MA", "Albany, NY", "Davenport, IA", "Plantation, FL", "Deltona, FL", "Federal Way, WA", "San Angelo, TX", "Tracy, CA", "Sunrise, FL"
]

CA_PROVINCES = ["Alberta", "British Columbia", "Ontario", "Quebec", "Manitoba", "New Brunswick", "Newfoundland and Labrador", "Nova Scotia", "Prince Edward Island", "Saskatchewan"]

CA_LOCATIONS = [ "Winnipeg, Manitoba", "Dieppe, New Brunswick", "Fredericton, New Brunswick", "Greater Lakeburn, New Brunswick", "Moncton, New Brunswick", "Saint John, New Brunswick", "Dartmouth, Nova Scotia", "Halifax, Nova Scotia", "Aurora, Ontario", "Barrie, Ontario", "Bowmanville, Ontario", "Chatham, Ontario", "Collingwood, Ontario", "East Gwillimbury, Ontario", "Goderich, Ontario", "Holland Landing, Ontario", "Leamington, Ontario", "London, Ontario", "Midland, Ontario", "Mississauga, Ontario", "Newmarket, Ontario", "Niagara Falls, Ontario", "North York, Ontario", "Oakville, Ontario", "Ottawa, Ontario", "Peterborough, Ontario", "Port Carling, Ontario", "Queensville, Ontario", "Richmond Hill, Ontario", "Seguin, Ontario", "St. Catharines, Ontario", "St. Marys, Ontario", "Sudbury, Ontario", "Thunder Bay, Ontario", "Toronto, Ontario", "Waterloo, Ontario", "Welland, Ontario", "Guelph, Ontario", "Scarborough, Ontario"]

LOCATIONS_TO_CHECK = (
    ["United States"]
    + US_STATES
    + US_CITIES
    + ["Canada"]
    + CA_PROVINCES
    + CA_LOCATIONS
)

# ==============================================================================
# SECTION 4: CORE LOGIC & HELPER FUNCTIONS
# ==============================================================================

def get_location_criterion_id(client, location_name):
    """Converts a location name into a Geo Target Constant Criterion ID."""
    gtc_service = client.get_service("GeoTargetConstantService")
    request = client.get_type("SuggestGeoTargetConstantsRequest")
    request.location_names.names.append(location_name)
    try:
        response = gtc_service.suggest_geo_target_constants(request)
        if not response.geo_target_constant_suggestions:
            print(f"    -> ⚠️  Could not find a location criterion for '{location_name}'.")
            return None
        return response.geo_target_constant_suggestions[0].geo_target_constant.resource_name
    except GoogleAdsException as ex:
        print(f"    -> ❌ Failed to get location ID for '{location_name}': {ex}")
        return None

def get_historical_metrics(client, customer_id, keywords, location_criterion):
    """
    Queries the Google Ads API for historical search volume data, starting from a fixed date (Jan 2023)
    up to the last full month.
    """
    print(f"  -> Fetching data for location: '{location_criterion}'...")

    request = client.get_type("GenerateKeywordHistoricalMetricsRequest")
    request.customer_id = customer_id
    request.keywords = keywords
    request.geo_target_constants.append(location_criterion)

    # --- UPDATED: Date Range Calculation ---
    # The end date is the last day of the previous month.
    today = datetime.date.today()
    last_full_month_date = today.replace(day=1) - relativedelta(days=1)
    end_year = last_full_month_date.year
    end_month_enum_value = getattr(client.get_type("MonthOfYearEnum").MonthOfYear, last_full_month_date.strftime('%B').upper())

    # The start date is now fixed to January 2023.
    start_year = 2023
    start_month_enum_value = client.get_type("MonthOfYearEnum").MonthOfYear.JANUARY

    print(f"    -> Requesting historical data from {start_year}-01 to {end_year}-{last_full_month_date.month:02d}.")

    request.historical_metrics_options.year_month_range.start.year = start_year
    request.historical_metrics_options.year_month_range.start.month = start_month_enum_value
    request.historical_metrics_options.year_month_range.end.year = end_year
    request.historical_metrics_options.year_month_range.end.month = end_month_enum_value

    try:
        time.sleep(1)
        keyword_plan_idea_service = client.get_service("KeywordPlanIdeaService")
        response = keyword_plan_idea_service.generate_keyword_historical_metrics(request=request)

        keyword_data = {}
        for result in response.results:
            monthly_volumes = {}
            for s in result.keyword_metrics.monthly_search_volumes:
                year = s.year
                month = s.month.value - 1

                if month == 0:
                    month = 12
                    year -= 1
                elif month > 12:
                    month -= 12
                    year += 1

                month_key = f"{year}-{month:02d}"
                monthly_volumes[month_key] = s.monthly_searches
            keyword_data[result.text] = monthly_volumes

        print(f"    -> Successfully fetched data for {len(keyword_data)} keywords.")
        return keyword_data
    except GoogleAdsException as ex:
        print(f'    -> ❌ Request failed: {ex.error.code().name}')
        for error in ex.failure.errors:
            print(f'       Error: {error.message} (Field: {error.location.field_path_elements[0].field_name})')
        return None
    except Exception as e:
        print(f"    -> ❌ An unexpected error occurred: {e}")
        return None

def aggregate_data(keyword_data, groups):
    """
    Aggregates search volume by the predefined groups (summing).
    This function now uses ALL unique months present in the API response data,
    with no upper limit on the number of months.
    """
    print("  -> Aggregating data...")

    # 1. Collect ALL unique YYYY-MM strings that actually exist in the API response.
    all_unique_months_from_api = set()
    for keyword_metrics in keyword_data.values():
        for month_key in keyword_metrics.keys():
            all_unique_months_from_api.add(month_key)

    # 2. Sort these months chronologically. This is crucial for correct sheet ordering.
    sorted_api_months = sorted(list(all_unique_months_from_api), key=lambda x: datetime.datetime.strptime(x, '%Y-%m'))

    # 3. Use all sorted months. The 25-month limit has been removed.
    final_months = sorted_api_months

    print(f"    -> Aggregating across {len(final_months)} total months.")

    # Initialize aggregated_volumes with the definitive month list (oldest to newest)
    aggregated_volumes = {group: {month: 0 for month in final_months} for group in groups}

    for group_name, keywords_in_group in groups.items():
        for keyword in keywords_in_group:
            if keyword in keyword_data:
                for month, volume in keyword_data[keyword].items():
                    # Check if the month is in our list (it always should be)
                    if month in final_months:
                        aggregated_volumes[group_name][month] += volume

    print("    -> Aggregation complete.")
    return aggregated_volumes, final_months

def update_google_sheet(spreadsheet_url, location_name, aggregated_data, months):
    """
    Updates the sheet by duplicating a template, clearing A1:X#, and pasting new data.
    Now dynamically adjusts for up to 25 months.
    """
    print(f"  -> Preparing to update sheet for '{location_name}'...")
    max_retries = 5
    for attempt in range(max_retries):
        try:
            spreadsheet = gc.open_by_url(spreadsheet_url)
            try:
                template_sheet = spreadsheet.worksheet("Sheet1")
            except gspread.exceptions.WorksheetNotFound:
                print("    -> ❌ CRITICAL ERROR: The template 'Sheet1' was not found."); return

            try:
                worksheet = spreadsheet.worksheet(location_name)
                print(f"    -> Found existing worksheet '{location_name}'.")
            except gspread.exceptions.WorksheetNotFound:
                print(f"    -> Worksheet '{location_name}' not found. Duplicating from 'Sheet1' at end of list...")

                source_sheet_id = template_sheet.id
                insert_index = len(spreadsheet.worksheets())

                requests = [{"duplicateSheet": {"sourceSheetId": source_sheet_id, "newSheetName": location_name, "insertSheetIndex": insert_index}}]
                spreadsheet.batch_update({"requests": requests})

                worksheet = spreadsheet.worksheet(location_name)
                print(f"    -> New worksheet '{location_name}' created and positioned at the end.")

            group_names = list(KEYWORD_GROUPS.keys())
            header_row = ['Keyword Group'] + months
            volume_rows = [[group] + [aggregated_data[group].get(month, 0) for month in months] for group in group_names]

            data_to_paste = [header_row] + volume_rows

            num_rows_to_clear = len(data_to_paste)
            num_cols_to_clear = len(data_to_paste[0])
            clear_end_cell = rowcol_to_a1(num_rows_to_clear, num_cols_to_clear)
            clear_range = f'A1:{clear_end_cell}'

            print(f"    -> Clearing target range: {clear_range}...")
            worksheet.batch_clear([clear_range])
            print("    -> Pasting monthly search volume data into A1.")
            worksheet.update('A1', data_to_paste, value_input_option='USER_ENTERED')

            worksheet.spreadsheet.batch_update({
                "requests": [{"autoResizeDimensions": {"dimensions": {"sheetId": worksheet.id, "dimension": "COLUMNS", "startIndex": 0, "endIndex": num_cols_to_clear}}}]
            })

            print(f"    -> ✅ Sheet '{location_name}' updated successfully!")
            return

        except gspread.exceptions.APIError as e:
            if e.response.status_code in [429, 500]:
                wait_time = 2 ** attempt
                print(f"    -> ⚠️ APIError (Status {e.response.status_code}). Retrying in {wait_time} seconds (attempt {attempt + 1}/{max_retries})...")
                time.sleep(wait_time)
            else:
                print(f"    -> ❌ An unrecoverable API error occurred while updating the Google Sheet: {e}")
                return
        except Exception as e:
            print(f"    -> ❌ An unexpected error occurred while updating the Google Sheet: {e}")
            return

    print(f"    -> ❌ Failed to update sheet '{location_name}' after {max_retries} attempts.")

def delete_all_except_specified_sheets(spreadsheet_url, sheets_to_keep):
    """
    Deletes all worksheets in a Google Sheet except those specified in sheets_to_keep.
    """
    print(f"\n--- Cleaning up Spreadsheet: Deleting unnecessary sheets ---")
    time.sleep(1)
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)
        all_worksheets = spreadsheet.worksheets()

        sheets_deleted = []
        for worksheet in all_worksheets:
            if worksheet.title not in sheets_to_keep:
                print(f"  -> Deleting sheet: '{worksheet.title}'...")
                spreadsheet.del_worksheet(worksheet)
                sheets_deleted.append(worksheet.title)
            else:
                print(f"  -> Keeping sheet: '{worksheet.title}'")

        if sheets_deleted:
            print(f"✅ Successfully deleted: {', '.join(sheets_deleted)}")
        else:
            print("  -> No sheets to delete (all specified sheets were kept).")

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"❌ Error: Spreadsheet not found at URL: {spreadsheet_url}. Cannot clean up.")
    except Exception as e:
        print(f"❌ An error occurred during sheet cleanup: {e}")

# --- NEW: Google Docs Logging Functions ---
# --- CORRECTED: Google Docs Logging Functions ---
def get_doc_content_length(service, document_id):
    """
    Gets the insertion point at the end of the Google Doc.
    The API's endIndex is an exclusive upper bound, so we subtract 1
    to get the last valid insertion index.
    """
    try:
        doc = service.documents().get(documentId=document_id, fields='body(content(endIndex))').execute()
        # FIX: Subtract 1 from the endIndex to get the correct insertion point.
        return doc['body']['content'][-1]['endIndex'] - 1
    except HttpError as e:
        print(f"    -> ❌ Error getting document length: {e}")
        # Fallback for empty or erroring doc: 1 is the beginning of the document.
        return 1

def batch_update_doc(service, document_id, requests):
    """Applies a batch of update requests to the Google Doc."""
    if not requests:
        print("  -> No log data to send to Google Docs.")
        return
    try:
        service.documents().batchUpdate(
            documentId=document_id,
            body={'requests': requests}
        ).execute()
        print("  -> ✅ Successfully wrote logs to Google Doc.")
    except HttpError as e:
        print(f"    -> ❌ Error writing to Google Doc: {e}")


# ==============================================================================
# SECTION 5: MAIN EXECUTION (UPDATED WITH H2 LOG HEADER)
# ==============================================================================
def main():
    print("\n--- Starting Main Execution ---")
    if not google_ads_client or not docs_service:
        print("\n❌ Halting execution: A required client (Google Ads or Docs) is not initialized.")
        return

    all_keywords = [kw for group in KEYWORD_GROUPS.values() for kw in group]
    if not all_keywords:
        print("No keywords found. Please add keywords.")
        return

    # --- Initialize Logging with H2 Header ---
    log_requests = []
    run_timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # We add a newline at the end for spacing after the header.
    header_text = f"{run_timestamp}\n"

    try:
        # Get the starting position for our new log entries.
        insert_at_index = get_doc_content_length(docs_service, DOCS_LOG_ID)-1
        if insert_at_index > 1: # Add a newline if we aren't at the very top
             header_text = "\n" + header_text
             insert_at_index += 1


        # 1. First request: Insert the header text.
        log_requests.append({
            'insertText': {
                'location': {'index': insert_at_index},
                'text': header_text
            }
        })

        # 2. Second request: Apply the 'Heading 2' style to the text we just inserted.
        log_requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': insert_at_index,
                    'endIndex': insert_at_index + len(header_text)
                },
                'paragraphStyle': {
                    'namedStyleType': 'HEADING_2'
                },
                # Specify the field we are updating.
                'fields': 'namedStyleType'
            }
        })

        # Send the batch containing both requests.
        batch_update_doc(docs_service, DOCS_LOG_ID, log_requests)

    except Exception as e:
        print(f"  -> ❌ Could not write initial header to Google Doc: {e}")


    # This list will hold the log strings for the current batch.
    log_batch = []

    # Optional: Enable sheet cleanup
    sheets_to_preserve = ['Sheet1', 'StateSummary']
    delete_all_except_specified_sheets(SPREADSHEET_URL, sheets_to_preserve)

    customer_id = credentials['login_customer_id'].replace("-", "")

    for i, location_name_str in enumerate(LOCATIONS_TO_CHECK):

        # Determine the fully qualified location name for the API
        if location_name_str in US_CITIES or location_name_str in CA_LOCATIONS:
            # Already fully qualified, e.g., "Los Angeles, CA" or "Toronto, Ontario"
            qualified_location_name = location_name_str
        elif location_name_str in US_STATES:
            # e.g., "California" -> "California, United States"
            qualified_location_name = f"{location_name_str}, United States"
        elif location_name_str in CA_PROVINCES:
            # e.g., "Ontario" -> "Ontario, Canada"
            qualified_location_name = f"{location_name_str}, Canada"
        elif location_name_str == "United States" or location_name_str == "Canada":
            # Whole country
            qualified_location_name = location_name_str
        else:
            # Should not be reached, but as a fallback, use the name as is
            qualified_location_name = location_name_str

        print(f"\n--- Processing Location ({i+1}/{len(LOCATIONS_TO_CHECK)}): {location_name_str} (as '{qualified_location_name}') ---")

        log_entry = f"Processing: {location_name_str}... "

        location_criterion_resource_name = get_location_criterion_id(google_ads_client, qualified_location_name)

        if location_criterion_resource_name:
            keyword_metrics = get_historical_metrics(google_ads_client, customer_id, all_keywords, location_criterion_resource_name)
            if keyword_metrics:
                aggregated_data, months = aggregate_data(keyword_metrics, KEYWORD_GROUPS)
                update_google_sheet(SPREADSHEET_URL, location_name_str, aggregated_data, months)
                log_entry += "✅ Success\n"
            else:
                print(f"  -> Skipping sheet update for '{location_name_str}' due to data fetch failure.")
                log_entry += "❌ Failed (Data Fetch Error)\n"
        else:
            print(f"  -> Skipping '{location_name_str}' due to invalid location name.")
            log_entry += "❌ Failed (Invalid Location)\n"

        log_batch.append(log_entry)

        if len(log_batch) >= 10:
            print("\n--- Writing batch of 10 logs to Google Doc ---")
            full_log_text = "".join(log_batch)
            try:
                insert_at_index = get_doc_content_length(docs_service, DOCS_LOG_ID)
                requests = [{'insertText': {'location': {'index': insert_at_index}, 'text': full_log_text}}]
                batch_update_doc(docs_service, DOCS_LOG_ID, requests)
                log_batch.clear()
            except Exception as e:
                print(f"  -> ❌ Could not write log batch to Google Doc: {e}")

    if log_batch:
        print("\n--- Writing final batch of logs to Google Doc ---")
        full_log_text = "".join(log_batch)
        try:
            insert_at_index = get_doc_content_length(docs_service, DOCS_LOG_ID)
            requests = [{'insertText': {'location': {'index': insert_at_index}, 'text': full_log_text}}]
            batch_update_doc(docs_service, DOCS_LOG_ID, requests)
        except Exception as e:
            print(f"  -> ❌ Could not write final log batch to Google Doc: {e}")

    print("\n\n--- All operations complete. ---")
    print(f"View the results here: {SPREADSHEET_URL}")

--- Initializing Google Ads Client ---
✅ Google Ads client initialized successfully!
--- Initializing Google Docs Client ---
✅ Google Docs client initialized successfully!


In [ ]:
if __name__ == "__main__":
    main()


--- Starting Main Execution ---
  -> ✅ Successfully wrote logs to Google Doc.

--- Cleaning up Spreadsheet: Deleting unnecessary sheets ---
  -> Keeping sheet: 'Sheet1'
  -> Keeping sheet: 'StateSummary'
  -> Deleting sheet: 'Clarksville, TN'...
  -> Deleting sheet: 'Ontario, CA'...
  -> Deleting sheet: 'Newport News, VA'...
  -> Deleting sheet: 'Elk Grove, CA'...
  -> Deleting sheet: 'Cary, NC'...
  -> Deleting sheet: 'Aurora, IL'...
  -> Deleting sheet: 'Salem, OR'...
  -> Deleting sheet: 'Pembroke Pines, FL'...
  -> Deleting sheet: 'Eugene, OR'...
  -> Deleting sheet: 'Santa Rosa, CA'...
  -> Deleting sheet: 'Rancho Cucamonga, CA'...
  -> Deleting sheet: 'Shreveport, LA'...
  -> Deleting sheet: 'Garden Grove, CA'...
  -> Deleting sheet: 'Oceanside, CA'...
  -> Deleting sheet: 'Fort Collins, CO'...
  -> Deleting sheet: 'Springfield, MO'...
  -> Deleting sheet: 'Murfreesboro, TN'...
  -> Deleting sheet: 'Surprise, AZ'...
  -> Deleting sheet: 'Lancaster, CA'...
  -> Deleting sheet: 'D

/tmp/ipython-input-2233666210.py:245: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_paste, value_input_option='USER_ENTERED')


Streaming output truncated to the last 5000 lines.
    -> Clearing target range: A1:AF6...
    -> Pasting monthly search volume data into A1.
    -> ✅ Sheet 'Pennsylvania' updated successfully!

--- Processing Location (40/397): Rhode Island (as 'Rhode Island, United States') ---
  -> Fetching data for location: 'geoTargetConstants/21172'...
    -> Requesting historical data from 2023-01 to 2025-07.
    -> Successfully fetched data for 58 keywords.
  -> Aggregating data...
    -> Aggregating across 31 total months.
    -> Aggregation complete.
  -> Preparing to update sheet for 'Rhode Island'...
    -> Worksheet 'Rhode Island' not found. Duplicating from 'Sheet1' at end of list...
    -> New worksheet 'Rhode Island' created and positioned at the end.
    -> Clearing target range: A1:AF6...
    -> Pasting monthly search volume data into A1.
    -> ✅ Sheet 'Rhode Island' updated successfully!

--- Writing batch of 10 logs to Google Doc ---
  -> ✅ Successfully wrote logs to Google Doc.

-

In [ ]:
# ==============================================================================
# SECTION 6: CUSTOM FUNCTIONS (State Summary - WITH CHANGE ANALYSIS)
# ==============================================================================

# IMPORTANT: Ensure SPREADSHEET_URL, KEYWORD_GROUPS, and US_STATES
# are defined in a previously executed cell before running this.

def create_state_summary_sheet(spreadsheet_url):
    "
    Creates/updates a 'StateSummary' sheet with states as rows and months as columns.
    It now intelligently appends summary columns for 1, 3, 6, 12, and 24-month
    change and percentage change, based on the available data.
    "
    print("\n--- Creating/Updating State Summary Sheet (with Change Analysis) ---")
    try:
        spreadsheet = gc.open_by_url(spreadsheet_url)

        try:
            summary_worksheet = spreadsheet.worksheet("StateSummary")
            print("  -> Found existing 'StateSummary' worksheet.")
        except gspread.exceptions.WorksheetNotFound:
            print("  -> 'StateSummary' worksheet not found. Creating it...")
            summary_worksheet = spreadsheet.add_worksheet(title="StateSummary", rows="100", cols="70", index=1)
            print("  -> New 'StateSummary' worksheet created.")

        months = []
        try:
            sample_sheet = spreadsheet.worksheet("United States")
            months = sample_sheet.row_values(1)[1:]
            if not months: raise ValueError("'United States' sheet is empty.")
            print(f"  -> Successfully loaded {len(months)} month headers from 'United States' sheet.")
        except (gspread.exceptions.WorksheetNotFound, ValueError, IndexError) as e:
            print(f"  -> WARNING: Could not get month headers ({e}). Generating fallback.")
            start_date = datetime.date(2023, 1, 1)
            end_date = datetime.date.today().replace(day=1) - relativedelta(days=1)
            current_date = start_date
            while current_date <= end_date:
                months.append(current_date.strftime('%Y-%m'))
                current_date += relativedelta(months=1)
            print(f"  -> Generated {len(months)} fallback month headers.")

        # --- Base data setup ---
        header_row = ['State'] + months
        data_rows = []
        all_sheet_titles = [s.title for s in spreadsheet.worksheets()]
        existing_state_sheets = [s for s in US_STATES if s in all_sheet_titles]

        if not existing_state_sheets:
            print("  -> No state-specific sheets found. Aborting summary creation.")
            return

        for state_name in existing_state_sheets:
            formula_row = [state_name]
            for j in range(len(months)):
                col_index = j + 2
                a1_notation = rowcol_to_a1(1, col_index)
                source_col_letter = ''.join(filter(str.isalpha, a1_notation))
                formula = f"='{state_name}'!{source_col_letter}7"
                formula_row.append(formula)
            data_rows.append(formula_row)

        # --- NEW: Add summary columns ---
        summary_periods = [('1m', 1), ('3m', 3), ('6m', 6), ('12m', 12), ('24m', 24)]
        num_months = len(months)
        last_month_col_index = 1 + num_months
        last_month_col_letter = ''.join(filter(str.isalpha, rowcol_to_a1(1, last_month_col_index)))

        percentage_format_requests = []

        for label, period_months in summary_periods:
            # Only add summary columns if we have enough historical data
            if num_months > period_months:
                print(f"  -> Adding {label} summary columns.")
                # Define new headers
                change_header = f"{label} Change"
                pct_change_header = f"{label} % Change"
                header_row.extend([change_header, pct_change_header])

                # Get the column for the base month (e.g., 12 months ago)
                base_month_col_index = last_month_col_index - period_months
                base_month_col_letter = ''.join(filter(str.isalpha, rowcol_to_a1(1, base_month_col_index)))

                # Get the column for the "Change" value we are about to add
                change_col_index = len(header_row) - 1
                change_col_letter = ''.join(filter(str.isalpha, rowcol_to_a1(1, change_col_index)))

                # Add the formulas to each data row
                for i, row in enumerate(data_rows):
                    sheet_row_num = i + 2 # +2 because sheet is 1-indexed and has a header

                    # Formula for absolute change: =(LastMonth - BaseMonth)
                    change_formula = f"={last_month_col_letter}{sheet_row_num}-{base_month_col_letter}{sheet_row_num}"

                    # Formula for % change: =IFERROR(Change / BaseMonth, 0)
                    pct_change_formula = f"=IFERROR({change_col_letter}{sheet_row_num}/{base_month_col_letter}{sheet_row_num}, 0)"

                    row.extend([change_formula, pct_change_formula])

                # Prepare a request to format the % change column
                pct_column_index = len(header_row) - 1 # 0-indexed column
                percentage_format_requests.append({
                    "repeatCell": {
                        "range": {"sheetId": summary_worksheet.id, "startColumnIndex": pct_column_index, "endColumnIndex": pct_column_index + 1},
                        "cell": {"userEnteredFormat": {"numberFormat": {"type": "PERCENT", "pattern": "0.00%"}}},
                        "fields": "userEnteredFormat.numberFormat"
                    }
                })

        # --- Combine all rows for final pasting ---
        final_rows_to_paste = [header_row] + data_rows

        # --- Update the sheet ---
        print(f"  -> Preparing to update sheet with {len(final_rows_to_paste)} rows and {len(header_row)} columns.")
        summary_worksheet.clear()
        summary_worksheet.update('A1', final_rows_to_paste, value_input_option='USER_ENTERED')

        # --- Apply all formatting requests and resize ---
        update_requests = [{
            "autoResizeDimensions": {"dimensions": {"sheetId": summary_worksheet.id, "dimension": "COLUMNS", "startIndex": 0, "endIndex": len(header_row)}}
        }] + percentage_format_requests

        summary_worksheet.spreadsheet.batch_update({"requests": update_requests})

        print("✅ 'StateSummary' sheet updated successfully with change analysis!")

    except Exception as e:
        print(f"❌ An error occurred while creating/updating the State Summary Sheet: {e}")


In [ ]:
# You would call this function in your main() after all individual location
# sheets have been processed. Example call in main():

create_state_summary_sheet(SPREADSHEET_URL)



--- Creating/Updating State Summary Sheet (with Change Analysis) ---
  -> Found existing 'StateSummary' worksheet.
  -> Successfully loaded 31 month headers from 'United States' sheet.
  -> Adding 1m summary columns.
  -> Adding 3m summary columns.
  -> Adding 6m summary columns.
  -> Adding 12m summary columns.
  -> Adding 24m summary columns.
  -> Preparing to update sheet with 51 rows and 42 columns.


/tmp/ipython-input-1105598732.py:115: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  summary_worksheet.update('A1', final_rows_to_paste, value_input_option='USER_ENTERED')


✅ 'StateSummary' sheet updated successfully with change analysis!


In [ ]:
# ==============================================================================
# SECTION 7: STANDALONE TEST FUNCTION (Save Raw API Response)
# ==============================================================================

# IMPORTANT: This function assumes your Google Ads API credentials
# (ADS_DEVELOPER_TOKEN, ADS_CLIENT_ID, etc.) are already set up in Colab's Secrets
# manager, as described in Section 2 of your main script.

# --- NEW/UPDATED IMPORTS REQUIRED ---
# No more json_format import needed for this simplified version
import json # Still needed if you use it elsewhere, but not for this specific dump
import datetime # Needed for date calculations
# ------------------------------------

def test_api_response_and_save(test_keyword: str, test_location_name: str, output_filename="raw_api_test_response.txt"): # Changed default extension to .txt
    "
    Makes a single Google Ads API call for a specific keyword and location,
    and saves the raw API response's string representation to a text file for debugging.

    Args:
        test_keyword (str): The single keyword to test.
        test_location_name (str): The location name (e.g., "United States", "Kansas").
        output_filename (str): The name of the text file to save the response to.
    "
    print(f"\n--- Running Standalone API Test for Keyword: '{test_keyword}' in '{test_location_name}' ---")

    google_ads_client_local = None
    try:
        credentials_local = {
            "developer_token": userdata.get('ADS_DEVELOPER_TOKEN'),
            "client_id": userdata.get('ADS_CLIENT_ID'),
            "client_secret": userdata.get('ADS_CLIENT_SECRET'),
            "refresh_token": userdata.get('ADS_REFRESH_TOKEN'),
            "login_customer_id": userdata.get('ADS_LOGIN_CUSTOMER_ID'),
            "use_proto_plus": True
        }
        google_ads_client_local = GoogleAdsClient.load_from_dict(credentials_local)
        print("  -> Google Ads client initialized for test.")
    except Exception as e:
        print(f"  ❌ Error initializing Google Ads client for test: {e}. Cannot proceed.")
        return

    customer_id_for_api = credentials_local['login_customer_id'].replace("-", "")

    # Get location criterion ID
    gtc_service = google_ads_client_local.get_service("GeoTargetConstantService")
    location_request = google_ads_client_local.get_type("SuggestGeoTargetConstantsRequest")
    location_request.location_names.names.append(test_location_name)

    location_criterion_resource_name = None
    try:
        location_response = gtc_service.suggest_geo_target_constants(location_request)
        if location_response.geo_target_constant_suggestions:
            location_criterion_resource_name = location_response.geo_target_constant_suggestions[0].geo_target_constant.resource_name
            print(f"  -> Found Criterion ID for '{test_location_name}': {location_criterion_resource_name}")
        else:
            print(f"  -> ⚠️ Could not find a location criterion for '{test_location_name}'. Cannot proceed.")
            return
    except GoogleAdsException as ex:
        print(f"  ❌ Failed to get location ID for '{test_location_name}': {ex.error.code().name}. Cannot proceed.")
        return
    except Exception as e:
        print(f"  ❌ An unexpected error occurred while getting location ID: {e}. Cannot proceed.")
        return

    # Define the historical metrics options to get the desired 25-month date range
    today = datetime.date.today()
    last_full_month_date = today.replace(day=1) - datetime.timedelta(days=1)
    end_year = last_full_month_date.year
    end_month_enum_value = getattr(google_ads_client_local.get_type("MonthOfYearEnum").MonthOfYear, last_full_month_date.strftime('%B').upper())

    start_dt = datetime.datetime(end_year, last_full_month_date.month, 1)
    for _ in range(24): # Go back 24 months from the end date for a total of 25 months
        year = start_dt.year
        month = start_dt.month - 1
        if month == 0:
            month = 12
            year -= 1
        start_dt = datetime.datetime(year, month, 1)

    start_year = start_dt.year
    start_month_enum_value = getattr(google_ads_client_local.get_type("MonthOfYearEnum").MonthOfYear, start_dt.strftime('%B').upper())

    print(f"  -> Requesting historical data from {start_year}-{start_dt.month:02d} to {end_year}-{last_full_month_date.month:02d} (total 25 months).")

    api_request = google_ads_client_local.get_type("GenerateKeywordHistoricalMetricsRequest")
    api_request.customer_id = customer_id_for_api
    api_request.keywords.append(test_keyword)
    api_request.geo_target_constants.append(location_criterion_resource_name)

    api_request.historical_metrics_options.year_month_range.start.year = start_year
    api_request.historical_metrics_options.year_month_range.start.month = start_month_enum_value
    api_request.historical_metrics_options.year_month_range.end.year = end_year
    api_request.historical_metrics_options.year_month_range.end.month = end_month_enum_value

    try:
        keyword_plan_idea_service = google_ads_client_local.get_service("KeywordPlanIdeaService")
        print(f"  -> Sending API request for keyword '{test_keyword}' in '{test_location_name}'...")
        raw_api_response = keyword_plan_idea_service.generate_keyword_historical_metrics(request=api_request)

        # --- FIX: Directly convert the raw response object to a string and save ---
        # This will save the __str__ representation (the one you saw printed to console).
        # It won't be proper JSON, but it will be the raw API response content.
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(str(raw_api_response)) # Convert to string and write directly

        print(f"  ✅ Raw API response (string representation) successfully saved to '{output_filename}'.")
        print(f"     You can view this file in Colab's file browser (left sidebar -> folder icon).")
        print("     NOTE: This content is the object's string representation, not formatted JSON.")
        print("\n--- Test complete. ---")

    except GoogleAdsException as ex:
        print(f"  ❌ API Request failed: {ex.error.code().name}.")
        for error in ex.failure.errors:
            print(f"    -> Error message: {error.message}")
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"      -> On field: {field_path_element.field_name}")
    except Exception as e:
        print(f"  ❌ An unexpected error occurred during API call: {e}")

# --- Example calls (uncomment to run in the cell) ---
# test_api_response_and_save("self storage unit", "United States")
# test_api_response_and_save("storage units", "Kansas")
# test_api_response_and_save("drive up storage", "Overland Park, Kansas", "drive_up_op_raw.txt")
